In [28]:
%load_ext tensorboard

In [32]:
import os
import datetime

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

import tensorflow as tf
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator

In [33]:
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [46]:
def build_network():
    classifier = tf.keras.models.Sequential()
    classifier.add(tf.keras.layers.Conv2D(32, (3, 3), input_shape=(64,64,3), activation='relu'))
    classifier.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))
    classifier.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu'))
    classifier.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))
    classifier.add(tf.keras.layers.Flatten())
    classifier.add(tf.keras.layers.Dense(128, activation='relu'))
    classifier.add(tf.keras.layers.Dense(1, activation='sigmoid'))
    classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return classifier

In [47]:
train_processor = ImageDataGenerator(rescale=1./255, # нормируем
                                     shear_range=0.2, # поворачиваем
                                     zoom_range=0.2, # зуммируем
                                     horizontal_flip=True) # рандомно транспорнируем?
test_processor = ImageDataGenerator(rescale=1./255)

In [48]:
os.listdir()

['.DS_Store',
 'dataset',
 'cnn.py',
 'logs',
 'cnn_with_keras.ipynb',
 '.ipynb_checkpoints',
 'cnn_homework_solution.py']

In [49]:
train = train_processor.flow_from_directory('dataset/training_set',
                                            target_size=(64,64),
                                            batch_size=32,
                                            class_mode='binary')
test = train_processor.flow_from_directory('dataset/test_set',
                                           target_size=(64,64),
                                           batch_size=32,
                                           class_mode='binary')

Found 8000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [50]:
classifier = build_network()

In [51]:
classifier.fit_generator(train,
                         steps_per_epoch=100,
                         epochs=25,
                         validation_data=test,
                         validation_steps=20,
                         callbacks=[tensorboard_callback])

Epoch 1/25
100/100 [==============================] - 41s 413ms/step - loss: 0.6977 - accuracy: 0.5056 - val_loss: 0.6914 - val_accuracy: 0.5219
Epoch 2/25
100/100 [==============================] - 46s 461ms/step - loss: 0.6799 - accuracy: 0.5713 - val_loss: 0.6667 - val_accuracy: 0.6125
Epoch 3/25
100/100 [==============================] - 46s 462ms/step - loss: 0.6558 - accuracy: 0.6175 - val_loss: 0.6525 - val_accuracy: 0.6281
Epoch 4/25
100/100 [==============================] - 36s 361ms/step - loss: 0.6205 - accuracy: 0.6622 - val_loss: 0.6484 - val_accuracy: 0.6156
Epoch 5/25
100/100 [==============================] - 33s 325ms/step - loss: 0.6046 - accuracy: 0.6678 - val_loss: 0.5777 - val_accuracy: 0.7047
Epoch 6/25
100/100 [==============================] - 32s 320ms/step - loss: 0.5978 - accuracy: 0.6694 - val_loss: 0.5764 - val_accuracy: 0.7078
Epoch 7/25
100/100 [==============================] - 32s 323ms/step - loss: 0.5963 - accuracy: 0.6772 - val_loss: 0.5718 - val_ac

In [19]:
test_image = image.load_img('dataset/single_prediction/cat_or_dog_1.jpg', target_size=(64,64))

In [20]:
test_image = image.img_to_array(test_image)

In [21]:
test_image.shape

(64, 64, 3)

In [22]:
test_image = np.expand_dims(test_image, axis=0) # добавляем размерность для батча

In [24]:
result = classifier.predict(test_image)

In [25]:
result

array([[1.]], dtype=float32)

In [26]:
train.class_indices

{'cats': 0, 'dogs': 1}

In [27]:
if result[0, 0] == 1:
    print('Dog')
else:
    print('Cat')

Dog
